## Prepare the environment

In [6]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Dec  6 15:57:21 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.17                 Driver Version: 546.17       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2070 ...  WDDM  | 00000000:01:00.0  On |                  N/A |
| 24%   31C    P5              24W / 215W |   1366MiB /  8192MiB |     29%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Update the unix package

In [7]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

'add-apt-repository' is not recognized as an internal or external command,
operable program or batch file.


'apt' is not recognized as an internal or external command,
operable program or batch file.
'apt' is not recognized as an internal or external command,
operable program or batch file.


## Install pip dependencies

In [8]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install hopsworks

  Cloning https://github.com/huggingface/transformers to c:\users\ merlijn\appdata\local\temp\pip-req-build-0oioe0rb
  Resolved https://github.com/huggingface/transformers to commit d6392482bd8af0d28903b5531ed04bd7273931a3
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8148332 sha256=81cea7a9a2b6f1df8b85b9a2e587e2a823c9544cc376eb4bc771ec40d2cb1677
  Stored in directory: C:\Users\ merlijn\AppData\Local\Temp\pip-ephem-wheel-cache-ki69k2vz\wheels\04\a3\f1\b88775f8e1665827525b19ac7590250f1038d947067beba9fb
Successfully built transformers


  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers 'C:\Users\ merlijn\AppData\Local\Temp\pip-req-build-0oioe0rb'
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\ merlijn\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\transformers\\models\\deprecated\\trajectory_transformer\\convert_trajectory_transformer_original_pytorch_checkpoint_to_pytorch.py'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



## Log into huggingface

##Log into hopsworks

In [9]:
import hopsworks
project = hopsworks.login()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/194708


## Load Dataset

In [18]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "it", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "it", split="test", use_auth_token=True)

print(common_voice)

You can remove this warning by passing 'token=<use_auth_token>' instead.


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 167606
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 15003
    })
})


# We remove accents, gender, age and other features that are not relevant for our purpose

In [19]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 167606
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 15003
    })
})


## Prepare Feature Extractor, Tokenizer and Data

### Load WhisperFeatureExtractor

In [20]:
!pip install transformers

  Using cached transformers-4.35.2-py3-none-any.whl.metadata (123 kB)
Using cached transformers-4.35.2-py3-none-any.whl (7.9 MB)


In [21]:
%pip install transformers

from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
preprocessor_config.json: 100%|██████████| 185k/185k [00:00<00:00, 13.6MB/s]
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development


### Load WhisperTokenizer

In [34]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Italian", task="transcribe")

### Combine To Create A WhisperProcessor

In [30]:
!pip install tensorflow
!pip install transformers==3.3.1

     ---------------------------------------- 0.0/57.5 kB ? eta -:--:--
     ---------------------------------------- 57.5/57.5 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/65.5 kB ? eta -:--:--
     ---------------------------------------- 65.5/65.5 kB 3.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     ------------ --------------------------- 0.5/1.5 MB 14.2 MB/s eta 0:00:01
     ------------------------- -------------- 1.0/1.5 MB 12.2 MB/s eta 0:00:01
     ---------------------------------------- 1.5/1.5 MB 11.8 MB/s eta 0:00:00
  Using cached wheel-0.42.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached MarkupSafe-2.1.3-cp311-cp311-win_amd64.whl.metadata (3.1 kB)
     ---------------------------------------- 0.0/151.7 kB ? eta -:--:--
     -------------------------------------- 151.7/151.7 kB 8.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/300.9 MB ? eta -:--:--
   ----------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
hsfs 3.4.5 requires markupsafe<2.1.0, but you have markupsafe 2.1.3 which is incompatible.


  Using cached transformers-3.3.1-py3-none-any.whl (1.1 MB)
  Using cached tokenizers-0.8.1rc2.tar.gz (97 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached sentencepiece-0.1.99-cp311-cp311-win_amd64.whl (977 kB)
  Using cached sacremoses-0.1.1-py3-none-any.whl.metadata (8.3 kB)
Using cached sacremoses-0.1.1-py3-none-any.whl (897 kB)
Failed to build tokenizers


  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [48 lines of output]
      C:\Users\ merlijn\AppData\Local\Temp\pip-build-env-yyiditce\overlay\Lib\site-packages\setuptools\dist.py:314: InformationOnly: Normalizing '0.8.1.rc2' to '0.8.1rc2'
        self.metadata.version = self._normalize_version(self.metadata.version)
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-311
      creating build\lib.win-amd64-cpython-311\tokenizers
      copying tokenizers\__init__.py -> build\lib.win-amd64-cpython-311\tokenizers
      creating build\lib.win-amd64-cpython-311\tokenizers\models
      copying tokenizers\models\__init__.py -> build\lib.win-amd64-cpython-311\tokenizers\models
      creating build\lib.win-amd64-cpython-311\tokenizers\decoders
      copying tokenizers\decoders\__init__.py -> build\lib.win-amd64-cpython-311\

In [31]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Italian", task="transcribe")

### Prepare Data

In [32]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [56]:
def prepare_dataset(batch, feature_extractor, tokenizer):
    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(batch["audio"]["array"], sampling_rate=batch["audio"]["sampling_rate"]).input_features[0]

    # Assuming you have a tokenizer defined somewhere
    # Replace 'tokenizer' with the actual name of your tokenizer
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch


# Assuming 'common_voice' is defined somewhere in your code
# Replace 'common_voice.column_names["train"]' with the actual columns you want to remove
common_voice = common_voice.map(
    lambda batch: prepare_dataset(batch, feature_extractor, tokenizer),
    remove_columns=common_voice.column_names["train"],
)

Map: 100%|██████████| 15003/15003 [06:53<00:00, 36.31 examples/s]


## Get information about the voices

In [57]:
help(common_voice)

Help on DatasetDict in module datasets.dataset_dict object:

class DatasetDict(builtins.dict)
 |  A dictionary (dict of str: datasets.Dataset) with dataset transforms methods (map, filter, etc.)
 |  
 |  Method resolution order:
 |      DatasetDict
 |      builtins.dict
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __getitem__(self, k) -> datasets.arrow_dataset.Dataset
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  align_labels_with_mapping(self, label2id: Dict, label_column: str) -> 'DatasetDict'
 |      Align the dataset's label ID and label name mapping to match an input `label2id` mapping.
 |      This is useful when you want to ensure that a model's predicted labels are aligned with the dataset.
 |      The alignment in done using the lowercase label names.
 |      
 |      Args:
 |          label2id (`dict`):
 |              The label name to ID mapping to align the dataset with.
 |          label_column (`str`):
 

In [ ]:
# Save the dataset locally
common_voice.save_to_disk("common_voice")